In [1]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
import os
import matplotlib.pyplot as plt
import numpy as np
import math

In [30]:
# number of classes
K = 4
 
# input 
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')

![image](https://user-images.githubusercontent.com/44194558/147188626-25937b87-d254-45e0-89f5-1d79d00e4d50.png)

In [31]:
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   

In [32]:
x = conv1_layer(input_tensor)
print(x.shape)

(None, 114, 114, 64)


In [33]:
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
    
    # Network output F(x)에 더해줌
    shortcut = x
    print('shorcut :', shortcut.shape)
    
    # 총 3개의 block으로 구성
    for i in range(3):
        if (i == 0):  # 각 stage마다 첫 번째 block은 이전 stage에서 받아온 tensor의 dimension을 증가시켜야 함
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            
            # Add 연산을 사용할 수 있도록 shortcut에 conv연산을 적용시켜 downsampling
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)    
            print('shorcut after 1x1 Conv :', shortcut.shape)        
            x = BatchNormalization()(x)
            print('F(x) :', x.shape)
            shortcut = BatchNormalization()(shortcut)
            
            # F(x) + x
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            print('F(x) + x :', x.shape)
            
            shortcut = x  # 이후 단계에서 shortcut은 언제나 256 채널로 유지됨
        
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
            
            # F(x) + x
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x        
    
    return x

In [34]:
x = conv2_layer(x)
print(x.shape)

shorcut : (None, 56, 56, 64)
shorcut after 1x1 Conv : (None, 56, 56, 256)
F(x) : (None, 56, 56, 256)
F(x) + x : (None, 56, 56, 256)
(None, 56, 56, 256)


In [36]:
def conv3_layer(x):        
    shortcut = x    
    print('shorcut :', shortcut.shape)
    
    # 4개 block
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            # downsampling
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            print('shorcut after 1x1 Conv :', shortcut.shape) 
            x = BatchNormalization()(x)
            print('F(x) :', x.shape)
            shortcut = BatchNormalization()(shortcut)            
            
            # skip-conn
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)  
            print('F(x) + x :', x.shape)  
 
            shortcut = x  # 이후 단계에서 계속 512 채널 유지 (downsampling 불필요)      
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x

In [37]:
x = conv3_layer(x)
print(x.shape)

shorcut : (None, 56, 56, 256)
shorcut after 1x1 Conv : (None, 28, 28, 512)
F(x) : (None, 28, 28, 512)
F(x) + x : (None, 28, 28, 512)
(None, 28, 28, 512)


In [38]:
def conv4_layer(x):
    shortcut = x    
    print('shorcut :', shortcut.shape)    
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            print('shorcut after 1x1 Conv :', shortcut.shape)
            x = BatchNormalization()(x)
            print('F(x) :', x.shape)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
            print('F(x) + x :', x.shape)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x

In [39]:
x = conv4_layer(x)
print(x.shape)

shorcut : (None, 28, 28, 512)
shorcut after 1x1 Conv : (None, 14, 14, 1024)
F(x) : (None, 14, 14, 1024)
F(x) + x : (None, 14, 14, 1024)
(None, 14, 14, 1024)


In [40]:
def conv5_layer(x):
    shortcut = x   
    print('shorcut :', shortcut.shape)  
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            print('shorcut after 1x1 Conv :', shortcut.shape)
            x = BatchNormalization()(x)
            print('F(x) :', x.shape)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
            print('F(x) + x :', x.shape)
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x

In [41]:
x = conv5_layer(x)
print(x.shape)

shorcut : (None, 14, 14, 1024)
shorcut after 1x1 Conv : (None, 7, 7, 2048)
F(x) : (None, 7, 7, 2048)
F(x) + x : (None, 7, 7, 2048)
(None, 7, 7, 2048)


In [42]:
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)

In [43]:
print(output_tensor.shape)

(None, 4)


In [44]:
resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_10 (ZeroPadding  (None, 230, 230, 3)  0          ['input[0][0]']                  
 2D)                                                                                              
                                                                                                  
 conv2d_29 (Conv2D)             (None, 112, 112, 64  9472        ['zero_padding2d_10[0][0]']      
                                )                                                             